In [ ]:
import pandas, json, numpy, requests, os, datetime, statistics, math, pytz, tweepy, sqlite3, time, re, random, matplotlib.pyplot as plt, sklearn, statsmodels.api as sm
from bs4 import BeautifulSoup
from scipy.signal import find_peaks

This script is for determining the patterns scoring:<br><br>
These results were obtained in the Stats_Linear_Regression_Fit_Moving_Average_Pattern_GITHUB notebook.<br><br>

Price Range:  0 - 10<br>
Below Intercept STD : 0.17513255566210528 %<br>
Above Intercept STD : 0.45301182526752415 %<br>
Average Below Intercept Percentage Difference : 0.34231092900939286 %<br>
Average Above Intercept Percentage Difference : 0.2519810678201617 %<br><br>
Price Range:  10 - 50<br>
Below Intercept STD : -0.36906552767945744 %<br>
Above Intercept STD : 0.10277267966445347 %<br>
Average Below Intercept Percentage Difference : 0.151529057215527 %<br>
Average Above Intercept Percentage Difference : 0.12988087673443394 %<br><br>
Price Range:  50 - 100<br>
Below Intercept STD : 0.03715929327731127 %<br>
Above Intercept STD : -0.12828675326439498 %<br>
Average Below Intercept Percentage Difference : 0.12860720001925952 %<br>
Average Above Intercept Percentage Difference : 0.12425878149198155 %<br><br>
Price Range:  100 - 200<br>
Below Intercept STD : 0.46246154188968047 %<br>
Above Intercept STD : 0.2685412082677696 %<br>
Average Below Intercept Percentage Difference : 0.11118525651283573 %<br>
Average Above Intercept Percentage Difference : 0.12508849534245137 %<br><br>
Price Range:  200 - 99999<br>
Below Intercept STD : 0.1989294081180204 %<br>
Above Intercept STD : 0.5199033509157069 %<br>
Average Below Intercept Percentage Difference : 0.1369846761425486 %<br>
Average Above Intercept Percentage Difference : 0.22152767365782866 %<br>

In [ ]:
def patterns_score(symbol, name):
    try:
        print(symbol)
        price_df = pandas.read_csv(os.getcwd() + "\\Daily Stock Prices\\" + symbol + ' - ' + name + '.csv', index_col= 'Date', parse_dates= True)
        price_df = price_df.sort_index(ascending=True)

        sma_30 = price_df.Close.rolling(window=30).mean()
        sma_60 = price_df.Close.rolling(window=60).mean()

        column_list = (list(price_df.columns))
        column_list.extend(['SMA_30', 'SMA_60'])

        price_df = price_df.merge(sma_30, on='Date').merge(sma_60, on='Date')

        price_df.columns = column_list

        patterns_score_list = []

        row_counter = 0

        for x, y in price_df.iterrows():
            patterns_score = 0

            if math.isnan(y.SMA_60) != True:
                below_difference = 1 - (y.SMA_30 / y.SMA_60)
                above_difference = 1 - (y.SMA_60 / y.SMA_30)
                sma_30_10_day_average_list = []
                sma_60_10_day_average_list = []

                for p in range(10):
                    sma_30_10_day_average_list.append(sma_30.iloc[row_counter-p])
                    sma_60_10_day_average_list.append(sma_60.iloc[row_counter-p])

                sma_30_10_day_average = numpy.nanmean(sma_30_10_day_average_list)
                sma_60_10_day_average = numpy.nanmean(sma_60_10_day_average_list)

                if y.SMA_30 < 10:
                    if y.SMA_30 <= y.SMA_60 and sma_30_10_day_average < sma_60_10_day_average:
                        if 0 <= below_difference <= (0.34231092900939286 - 0.17513255566210528) / 100:
                            patterns_score = 500
                        elif (0.34231092900939286 - 0.17513255566210528) / 100 < below_difference <= (0.34231092900939286) / 100:
                            patterns_score = 300
                        elif (0.34231092900939286) / 100 < below_difference <= (0.34231092900939286 + 0.17513255566210528) / 100:
                            patterns_score = 150

                    elif y.SMA_30 >= y.SMA_60 and sma_30_10_day_average > sma_60_10_day_average:
                        if 0 <= above_difference <= (0.2519810678201617) / 100:
                            patterns_score = -500
                        elif (0.2519810678201617) / 100 < above_difference <= (0.2519810678201617 + (0.45301182526752415 / 2)) / 100:
                            patterns_score = -300
                        elif (0.2519810678201617 + (0.45301182526752415 / 2)) / 100 < above_difference <= (0.2519810678201617 + 0.45301182526752415) / 100:
                            patterns_score = -150

                elif 10 <= y.SMA_30 < 50:
                    if y.SMA_30 <= y.SMA_60 and sma_30_10_day_average < sma_60_10_day_average:
                        if 0 <= below_difference <= (0.151529057215527) / 100:
                            patterns_score = 500
                        elif (0.151529057215527) / 100 < below_difference <= (0.151529057215527 * 2) / 100:
                            patterns_score = 300

                    elif y.SMA_30 >= y.SMA_60 and sma_30_10_day_average > sma_60_10_day_average:
                        if 0 <= above_difference <= (0.12988087673443394 - 0.10277267966445347) / 100:
                            patterns_score = -500
                        elif (0.12988087673443394 - 0.10277267966445347) / 100 < above_difference <= (0.12988087673443394) / 100:
                            patterns_score = -300
                        elif (0.12988087673443394) / 100 < above_difference <= (0.12988087673443394 + 0.10277267966445347) / 100:
                            patterns_score = -150    

                elif 50 <= y.SMA_30 < 100:
                    if y.SMA_30 <= y.SMA_60 and sma_30_10_day_average < sma_60_10_day_average:
                        if 0 <= below_difference <= (0.12860720001925952 - 0.03715929327731127) / 100:
                            patterns_score = 500
                        elif (0.12860720001925952 - 0.03715929327731127) / 100 < below_difference <= (0.12860720001925952) / 100:
                            patterns_score = 300
                        elif (0.12860720001925952) / 100 < below_difference <= (0.12860720001925952 + 0.03715929327731127) / 100:
                            patterns_score = 150

                    elif y.SMA_30 >= y.SMA_60 and sma_30_10_day_average > sma_60_10_day_average:
                        if 0 <= above_difference <= (0.12425878149198155) / 100:
                            patterns_score = -500
                        elif (0.12425878149198155) / 100 < above_difference <= (0.12425878149198155 + (0.12425878149198155 / 2)) / 100:
                            patterns_score = -300
                        elif (0.12425878149198155 + (0.12425878149198155 / 2)) / 100 < above_difference <= (0.12425878149198155 + 0.12425878149198155) / 100:
                            patterns_score = -150

                elif 100 <= y.SMA_30 < 200:
                    if y.SMA_30 <= y.SMA_60 and sma_30_10_day_average < sma_60_10_day_average:
                        if 0 <= below_difference <= (0.11118525651283573 - 0.46246154188968047/6) / 100:
                            patterns_score = 500
                        elif (0.11118525651283573 - 0.46246154188968047/6) / 100 < below_difference <= (0.11118525651283573) / 100:
                            patterns_score = 300
                        elif (0.11118525651283573) / 100 < below_difference <= (0.11118525651283573 + 0.46246154188968047) / 100:
                            patterns_score = 150

                    elif y.SMA_30 >= y.SMA_60 and sma_30_10_day_average > sma_60_10_day_average:
                        if 0 <= above_difference <= (0.12508849534245137 - 0.2685412082677696/6) / 100:
                            patterns_score = -500
                        elif (0.12508849534245137 - 0.2685412082677696/6) / 100 < above_difference <= (0.12508849534245137) / 100:
                            patterns_score = -300
                        elif (0.12508849534245137) / 100 < above_difference <= (0.12508849534245137 + 0.2685412082677696) / 100:
                            patterns_score = -150

                elif y.SMA_30 >= 200:
                    if y.SMA_30 <= y.SMA_60 and sma_30_10_day_average < sma_60_10_day_average:
                        if 0 <= below_difference <= (0.1369846761425486 - 0.1989294081180204/10) / 100:
                            patterns_score = 500
                        elif (0.1369846761425486 - 0.1989294081180204/10) / 100 < below_difference <= (0.1369846761425486) / 100:
                            patterns_score = 300
                        elif (0.1369846761425486) / 100 < below_difference <= (0.1369846761425486 + 0.1989294081180204) / 100:
                            patterns_score = 150

                    elif y.SMA_30 >= y.SMA_60 and sma_30_10_day_average > sma_60_10_day_average:
                        if 0 <= above_difference <= (0.22152767365782866 - 0.5199033509157069/10) / 100:
                            patterns_score = -500
                        elif (0.22152767365782866 - 0.5199033509157069/10) / 100 < above_difference <= (0.22152767365782866) / 100:
                            patterns_score = -300
                        elif (0.22152767365782866) / 100 < above_difference <= (0.22152767365782866 + 0.5199033509157069) / 100:
                            patterns_score = -150  

            row_counter += 1

            patterns_score_list.append(patterns_score)

        price_df['Patterns_Score'] = patterns_score_list  

        row_counter = 0

        for x, y in price_df.iterrows():
            if y.Patterns_Score < 0:
                try:
                    for r in range(1, 11):
                        if price_df.iloc[row_counter + r]['Close'] > y.Close: 
                            continue
                        elif price_df.iloc[row_counter + r]['Close'] < y.Close:
                            price_df['Patterns_Score'][price_df.index == price_df.index[row_counter + r]] = y.Patterns_Score/(r * 1.5)
                except Exception:
                    pass

            elif y.Patterns_Score > 0:
                try:
                    for r in range(1, 10):
                        if price_df.iloc[row_counter + r]['Close'] < y.Close:
                            continue
                        elif price_df.iloc[row_counter + r]['Close'] > y.Close:
                            price_df['Patterns_Score'][price_df.index == price_df.index[row_counter + r]] = y.Patterns_Score/(r * 1.5) 
                except Exception:
                    pass

            row_counter += 1

        if not os.path.exists(os.getcwd() + '\\Patterns Score\\'):
            os.makedirs(os.getcwd() + '\\Patterns Score\\')

        price_df.to_csv(os.getcwd() + '\\Patterns Score\\'  + symbol + ' - ' + name + ' - Patterns Score.csv')

    except Exception:
        print('Error: ', symbol)
        pass


In [ ]:
### Looping:
stocks_and_names_with_indices = pandas.read_csv('merged_NYSE_AMEX_removed_intercept_pattern.csv')

for x, y in stocks_and_names_with_indices.iterrows():
    try:
        patterns_score(symbol= y['Symbol'], name= y['Description'])
    except Exception:
        pass
    